In [ ]:
# For handling the csv file
import pandas as pd

# Helps in text processing (e.g., removing special characters or extracting patterns from text).
import re

# Converts text into numerical values using TF-IDF 
# helping the model understand the importance of words.
from sklearn.feature_extraction.text import TfidfVectorizer

# Splits data into training and testing sets to evaluate model performance
from sklearn.model_selection import train_test_split

# A type of deep learning model where layers are stacked one after another
from tensorflow.keras.models import Sequential

# Dense: A fully connected layer where each neuron is connected to the previous layer.
# Dropout: Prevents overfitting by randomly turning off some neurons during training.
# BatchNormalization: Normalizes inputs to speed up training and improve stability.
# LeakyReLU: An activation function that helps the model learn complex patterns.
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU

# Helps in adjusting model parameters efficiently during training.
from tensorflow.keras.optimizers import Adam

# To save the functions
import pickle

In [ ]:
# Read the CSV containing the data
df = pd.read_csv("./Mental_health_data.csv")

# Get the information and the first 5 rows
df.info(), df.head()

In [ ]:
# Add the title to the text if the text is missing
df["text"] = df["text"].combine_first(df["title"])

# Combine 'title' and 'text' fields for training
df["problem_description"] = df["title"] + " " + df["text"]

# Drop the Unnamed: 0 column, the title and the text field
df.drop(columns=["Unnamed: 0", "title", "text"], inplace=True)

# Verify if there are still missing values
missing_values = df.isnull().sum()
print(missing_values)

In [ ]:
# Define a small set of words that does not effect the model
stopwords = set([
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours",
    "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself",
    "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this",
    "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have",
    "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if",
    "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against",
    "between", "into", "through", "during", "before", "after", "above", "below", "to", "from",
    "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once"
])

# Function to preprocess the text
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\W+', ' ', text)  # Remove special characters & punctuation
    words = text.split()  # Tokenization
    words = [word for word in words if word not in stopwords]  # Remove stopwords
    return " ".join(words)

# Apply preprocessing
df["problem_description"] = df["problem_description"].apply(preprocess_text)

# Check for the cleaned data
display(df.head())

In [ ]:
x = df["problem_description"]
y = df["target"]

# Convert combined text into numerical representation using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # Limit features for efficiency
x_tfidf = vectorizer.fit_transform(x) 

# Save the TF-IDF vectorizer
with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_tfidf, y, test_size=0.2, random_state=42)

In [ ]:
# Define a ANN model
# The input shape is the number of input to be given and num_classes is the number of output classes
def build_improved_ann(input_shape, num_classes):
    # Sequential Model ensures layers are stacked in order
    model = Sequential([ 
        # First Hidden Layer
        Dense(512, input_shape=(input_shape,), kernel_initializer='he_normal'), # 512 neurons, 
        BatchNormalization(), # Normalizes activations to stabilize learning and speed up convergence.
        LeakyReLU(), # A variation of ReLU that allows small negative values, avoiding dead neurons.
        Dropout(0.5), # Randomly drops 50% of neurons during training to prevent overfitting.
        
        Dense(256, kernel_initializer='he_normal'),
        BatchNormalization(),
        LeakyReLU(),
        Dropout(0.5),
        
        Dense(128, kernel_initializer='he_normal'),
        BatchNormalization(),
        LeakyReLU(),
        Dropout(0.4),
        
        Dense(64, kernel_initializer='he_normal'),
        BatchNormalization(),
        LeakyReLU(),
        Dropout(0.3),
        
        Dense(num_classes, activation='softmax')  # Assuming multi-class classification
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model

# Example usage (replace with actual dataset input shape and classes)
input_shape = 5000
num_classes = 5  # Example number of target classes, update accordingly

ann_model = build_improved_ann(input_shape, num_classes)
ann_model.summary()

# Train the model
history = ann_model.fit(
    x_train.toarray(), y_train, 
    validation_data=(x_test.toarray(), y_test),
    epochs=50, 
    batch_size=64, 
)

# Evaluate the model
loss, accuracy = ann_model.evaluate(x_test.toarray(), y_test)
print(f"Test Accuracy: {accuracy:.4f}")

In [ ]:
# Save the trained model
ann_model.save("mental_health_ann_model.keras")